In [1]:
%matplotlib inline
import pandas as pd
import csv as csv
import numpy as np
import xgboost as xgb
import scipy

I set the geometric mean per store per day of the week as a benchmark for my model.

In [2]:
#Loading data
ds    = pd.read_csv('../csv/train.csv', header=0, dtype={'StateHoliday':'str'})
store = pd.read_csv('../csv/store.csv', header=0)
test  = pd.read_csv('../csv/test.csv', header=0, dtype={'StateHoliday':'str'})

In [6]:
#Constructing dataframe with geometric mean
geomean_benchmark = pd.concat(
    [ds[(ds.Sales!=0)&(ds.DayOfWeek!=7)].groupby(['Store','DayOfWeek']).agg({
                'Sales':lambda x: np.exp(scipy.stats.mstats.gmean(np.log(x+1)))-1
            }),
    ds[ds.DayOfWeek==7].groupby(['Store','DayOfWeek']).agg({
                'Sales':lambda x: np.exp(scipy.stats.mstats.gmean(np.log(x+1)))-1
            })
    ]
).reset_index()

In [7]:
geomean_benchmark = pd.merge(geomean_benchmark,test,on=['Store','DayOfWeek'])

In [8]:
len(geomean_benchmark)

41088

In [9]:
#Setting solution dataframe
sol = pd.DataFrame({'Id':geomean_benchmark['Id'],'Sales':geomean_benchmark['Sales']})

In [10]:
len(sol)

41088

In [11]:
#Writing file to submit to kaggle with geometric mean benchmark
sol.to_csv('sol_GM.csv',index=False)